In [1]:
import numpy as np
import os
import pickle

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Concatenate
from keras.layers import Embedding, LSTM, Bidirectional, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import LearningRateScheduler as LRS
from keras.models import load_model
from keras.utils import plot_model

import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import string
import re

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from transformers import TFBertForSequenceClassification

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [3]:
def configure_callbacks(model_id, kpi_to_monitor='val_accuracy'):
    # Without log/ or models/ subfolder as not possible to access unexisting folders
    # If possible to train with jupyter revise
    path = "models"
    name = "ap"
    
    log_filename = '%s/%s-%s.log' % (path,name, model_id)
    csv_logger = CSVLogger(log_filename)
    
    chk_1_model_filename = '%s/%s-%s-{epoch:04d}-{%s:.6f}.h5' % (path, name, model_id, kpi_to_monitor)
    chk_2_model_filename = '%s/%s-%s.h5' % (path, name, model_id)
    
    # Save best model fully not only weights after each epoch (period=1) 
    # with best accuracy value (mode=max, save_best_only=True)
    checkpoint1 = ModelCheckpoint(
        chk_1_model_filename,
        monitor=kpi_to_monitor,
        save_best_only=True,
        save_weights_only=False, 
        verbose=1, mode='max', period=1
    )

    checkpoint2 = ModelCheckpoint(
        chk_2_model_filename, 
        monitor=kpi_to_monitor,
        save_best_only=False,
        save_weights_only=False, 
        verbose=1, mode='auto', period=1
    )

    callbacks = [csv_logger, checkpoint1, checkpoint2]
    return callbacks

In [4]:
######### AUTHOR PROFILING FUNCTION FOR JOINING PREDICTIONS #########

# de 20 0s y 20 1s

def author_profiling_report(author_predictions, number_authors=40):
    n = int(number_authors/2)
    a = np.zeros(n)
    b = np.ones(n)        
    author_profile = np.concatenate([a,b])
    
    # Check author profiling -> 8,000 predictions
    # Split into 40 authors -> 200 tweets per author
    author_predictions = np.average(np.array_split(author_predictions, number_authors), axis=1)
    author_predictions = np.array([1 if ap >= 0.5 else 0 for ap in author_predictions])

    print(classification_report(author_profile, author_predictions, labels=[0, 1], target_names=['not hate','hate']))
    return author_predictions

### Load Data

In [5]:
pickle_file = open('es_indv.pickle', 'rb')
es_indv = pickle.load(pickle_file)

embedding_matrix_es_indv = es_indv[0]
train_padded_es_indv, y_train_es_indv = es_indv[1], es_indv[2]
valid_padded_es_indv, y_valid_es_indv = es_indv[3], es_indv[4]
test_padded_es_indv = es_indv[5]

In [6]:
pickle_file = open('en_indv.pickle', 'rb')
en_indv = pickle.load(pickle_file)

embedding_matrix_en_indv = en_indv[0]
train_padded_en_indv, y_train_en_indv = en_indv[1], en_indv[2]
valid_padded_en_indv, y_valid_en_indv = en_indv[3], en_indv[4]
test_padded_en_indv = en_indv[5]

In [7]:
pickle_file = open('es_20.pickle', 'rb')
es_20 = pickle.load(pickle_file)

train_padded_es_20, y_train_es_20 = es_20[0], es_20[1]
valid_padded_es_20, y_valid_es_20 = es_20[2], es_20[3]
test_padded_es_20 = es_20[4]

In [8]:
pickle_file = open('en_20.pickle', 'rb')
en_20 = pickle.load(pickle_file)

train_padded_en_20, y_train_en_20 = en_20[0], en_20[1]
valid_padded_en_20, y_valid_en_20 = en_20[2], en_20[3]
test_padded_en_20 = en_20[4]

In [9]:
pickle_file = open('es_cml.pickle', 'rb')
es_cml = pickle.load(pickle_file)

x_train_es, y_train_es = es_cml[0], es_cml[1]
x_valid_es, y_valid_es = es_cml[2], es_cml[3]
x_test_es = es_cml[4]

In [10]:
pickle_file = open('en_cml.pickle', 'rb')
en_cml = pickle.load(pickle_file)

x_train_en, y_train_en = en_cml[0], en_cml[1]
x_valid_en, y_valid_en = en_cml[2], en_cml[3]
x_test_en = en_cml[4]

In [17]:
pickle_file = open('es_indv_bert.pickle', 'rb')
es_indv_bert = pickle.load(pickle_file)

train_padded_es_indv_bert, train_mask_es_indv, y_train_es = es_indv_bert[0], es_indv_bert[1], es_indv_bert[2]
valid_padded_es_indv_bert, valid_mask_es_indv, y_valid_es = es_indv_bert[3], es_indv_bert[4], es_indv_bert[5]
test_padded_es_indv_bert, test_mask_es_indv = es_indv_bert[6], es_indv_bert[7]

In [18]:
pickle_file = open('es_20_bert.pickle', 'rb')
es_20_bert = pickle.load(pickle_file)

train_padded_es_20_bert, train_mask_es_20, y_train_es = es_20_bert[0], es_20_bert[1], es_20_bert[2]
valid_padded_es_20_bert, valid_mask_es_20, y_valid_es = es_20_bert[3], es_20_bert[4], es_20_bert[5]
test_padded_es_20_bert, test_mask_es_20 = es_20_bert[6], es_20_bert[7]

### RNN Models

#### Individual tweets

In [6]:
# ES INDVIDUAL TWEETS - PRETRAINED EMB
max_words = 10000
word_embedding_size = 300
max_length = 15

model1 = Sequential()
model1.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    weights=[embedding_matrix_es_indv], # embeddings
                    input_length=max_length,
                    trainable=False,
                    mask_zero=True))
model1.add(LSTM(units=128))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 300)           3000000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,219,777
Trainable params: 219,777
Non-trainable params: 3,000,000
_________________________________________________________________


In [8]:
model1.fit(x=train_padded_es_indv,
          y=y_train_es_indv,
          validation_data=(valid_padded_es_indv, y_valid_es_indv),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model1.predict(valid_padded_es_indv)
author_profiling_report(predictions)

Epoch 1/25
500/500 [==============================] - 5s 10ms/step - loss: 0.6459 - accuracy: 0.6225 - val_loss: 0.6677 - val_accuracy: 0.5984
Epoch 2/25
500/500 [==============================] - 5s 10ms/step - loss: 0.6414 - accuracy: 0.6263 - val_loss: 0.6634 - val_accuracy: 0.6168
Epoch 3/25
500/500 [==============================] - 5s 11ms/step - loss: 0.6352 - accuracy: 0.6346 - val_loss: 0.6615 - val_accuracy: 0.6090
Epoch 4/25
500/500 [==============================] - 5s 10ms/step - loss: 0.6310 - accuracy: 0.6389 - val_loss: 0.6691 - val_accuracy: 0.6070
Epoch 5/25
500/500 [==============================] - 5s 10ms/step - loss: 0.6270 - accuracy: 0.6474 - val_loss: 0.6619 - val_accuracy: 0.6047
Epoch 6/25
500/500 [==============================] - 5s 10ms/step - loss: 0.6252 - accuracy: 0.6448 - val_loss: 0.6606 - val_accuracy: 0.6169
Epoch 7/25
500/500 [==============================] - 5s 10ms/step - loss: 0.6207 - accuracy: 0.6518 - val_loss: 0.6687 - val_accuracy: 0.6056

In [9]:
# ES INDVIDUAL TWEETS - PRETRAINED EMB TRAINABLE
max_words = 10000
word_embedding_size = 300
max_length = 15

model2 = Sequential()
model2.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    weights=[embedding_matrix_es_indv], # embeddings
                    input_length=max_length,
                    trainable=True,
                    mask_zero=True))
model2.add(LSTM(units=128))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 300)           3000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 3,219,777
Trainable params: 3,219,777
Non-trainable params: 0
_________________________________________________________________


In [10]:
model2.fit(x=train_padded_es_indv,
          y=y_train_es_indv,
          validation_data=(valid_padded_es_indv, y_valid_es_indv),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model2.predict(valid_padded_es_indv)
author_profiling_report(predictions)

Epoch 1/25
500/500 [==============================] - 15s 30ms/step - loss: 0.6300 - accuracy: 0.6343 - val_loss: 0.6731 - val_accuracy: 0.6060
Epoch 2/25
500/500 [==============================] - 14s 28ms/step - loss: 0.4900 - accuracy: 0.7618 - val_loss: 0.6989 - val_accuracy: 0.6112
Epoch 3/25
500/500 [==============================] - 14s 28ms/step - loss: 0.3905 - accuracy: 0.8190 - val_loss: 0.8267 - val_accuracy: 0.5989
Epoch 4/25
500/500 [==============================] - 14s 28ms/step - loss: 0.3098 - accuracy: 0.8591 - val_loss: 1.0365 - val_accuracy: 0.5885
Epoch 5/25
500/500 [==============================] - 14s 28ms/step - loss: 0.2417 - accuracy: 0.8932 - val_loss: 1.2318 - val_accuracy: 0.5864
Epoch 6/25
500/500 [==============================] - 14s 28ms/step - loss: 0.1867 - accuracy: 0.9196 - val_loss: 1.5532 - val_accuracy: 0.5781
Epoch 7/25
500/500 [==============================] - 15s 30ms/step - loss: 0.1462 - accuracy: 0.9357 - val_loss: 1.7653 - val_accuracy:

In [11]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB
max_words = 10000
word_embedding_size = 300
max_length = 15

model3 = Sequential()
model3.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_es_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model3.add(LSTM(units=128))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 300)           3000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,219,777
Trainable params: 3,219,777
Non-trainable params: 0
_________________________________________________________________


In [12]:
model3.fit(x=train_padded_es_indv,
          y=y_train_es_indv,
          validation_data=(valid_padded_es_indv, y_valid_es_indv),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model3.predict(valid_padded_es_indv)
author_profiling_report(predictions)

Epoch 1/25
500/500 [==============================] - 16s 32ms/step - loss: 0.6093 - accuracy: 0.6670 - val_loss: 0.6463 - val_accuracy: 0.6360
Epoch 2/25
500/500 [==============================] - 15s 31ms/step - loss: 0.4912 - accuracy: 0.7613 - val_loss: 0.6960 - val_accuracy: 0.6210
Epoch 3/25
500/500 [==============================] - 15s 30ms/step - loss: 0.3928 - accuracy: 0.8133 - val_loss: 0.8568 - val_accuracy: 0.6104
Epoch 4/25
500/500 [==============================] - 15s 31ms/step - loss: 0.3115 - accuracy: 0.8546 - val_loss: 1.1021 - val_accuracy: 0.5999
Epoch 5/25
500/500 [==============================] - 15s 30ms/step - loss: 0.2449 - accuracy: 0.8865 - val_loss: 1.3074 - val_accuracy: 0.5994
Epoch 6/25
500/500 [==============================] - 15s 30ms/step - loss: 0.1932 - accuracy: 0.9117 - val_loss: 1.5461 - val_accuracy: 0.5924
Epoch 7/25
500/500 [==============================] - 15s 31ms/step - loss: 0.1539 - accuracy: 0.9308 - val_loss: 1.8436 - val_accuracy:

In [13]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB BI
max_words = 10000
word_embedding_size = 300
max_length = 15

model4 = Sequential()
model4.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model4.add(Bidirectional(LSTM(units=128, dropout=0.05, recurrent_dropout=0.2)))
model4.add(Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 15, 300)           3000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               439296    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 3,439,553
Trainable params: 3,439,553
Non-trainable params: 0
_________________________________________________________________


In [14]:
model4.fit(x=train_padded_es_indv,
          y=y_train_es_indv,
          validation_data=(valid_padded_es_indv, y_valid_es_indv),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model4.predict(valid_padded_es_indv)
author_profiling_report(predictions)

Epoch 1/25
500/500 [==============================] - 37s 73ms/step - loss: 0.6056 - accuracy: 0.6709 - val_loss: 0.6424 - val_accuracy: 0.6382
Epoch 2/25
500/500 [==============================] - 36s 72ms/step - loss: 0.4906 - accuracy: 0.7616 - val_loss: 0.6902 - val_accuracy: 0.6208
Epoch 3/25
500/500 [==============================] - 36s 72ms/step - loss: 0.3831 - accuracy: 0.8242 - val_loss: 0.7782 - val_accuracy: 0.6033
Epoch 4/25
500/500 [==============================] - 36s 72ms/step - loss: 0.2830 - accuracy: 0.8733 - val_loss: 1.0891 - val_accuracy: 0.5899
Epoch 5/25
500/500 [==============================] - 36s 72ms/step - loss: 0.2025 - accuracy: 0.9088 - val_loss: 1.3936 - val_accuracy: 0.5896
Epoch 6/25
500/500 [==============================] - 36s 72ms/step - loss: 0.1465 - accuracy: 0.9371 - val_loss: 1.7412 - val_accuracy: 0.5859
Epoch 7/25
500/500 [==============================] - 36s 72ms/step - loss: 0.1105 - accuracy: 0.9537 - val_loss: 2.1149 - val_accuracy:

In [6]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET
max_words = 10000
word_embedding_size = 300
max_length = 15

model7 = Sequential()
model7.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model7.add(LSTM(units=128, return_sequences=True))
model7.add(LSTM(units=128, return_sequences=True))
model7.add(LSTM(units=128, return_sequences=True))
model7.add(LSTM(units=128))
model7.add(Dense(1, activation='sigmoid'))

model7.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model7.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 300)           3000000   
_________________________________________________________________
lstm (LSTM)                  (None, 15, 128)           219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 128)           131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,614,529
Trainable params: 3,614,529
Non-trainable params: 0
______________________________________________

In [7]:
model7.fit(x=train_padded_es_indv,
          y=y_train_es_indv,
          validation_data=(valid_padded_es_indv, y_valid_es_indv),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model7.predict(valid_padded_es_indv)
author_profiling_report(predictions)

Epoch 1/25
500/500 [==============================] - 27s 54ms/step - loss: 0.6173 - accuracy: 0.6593 - val_loss: 0.6624 - val_accuracy: 0.6321
Epoch 2/25
500/500 [==============================] - 22s 44ms/step - loss: 0.5071 - accuracy: 0.7532 - val_loss: 0.6862 - val_accuracy: 0.6198
Epoch 3/25
500/500 [==============================] - 22s 44ms/step - loss: 0.4177 - accuracy: 0.8063 - val_loss: 0.7704 - val_accuracy: 0.6125
Epoch 4/25
500/500 [==============================] - 23s 46ms/step - loss: 0.3297 - accuracy: 0.8486 - val_loss: 0.9657 - val_accuracy: 0.6031
Epoch 5/25
500/500 [==============================] - 23s 45ms/step - loss: 0.2583 - accuracy: 0.8825 - val_loss: 1.1541 - val_accuracy: 0.5919
Epoch 6/25
500/500 [==============================] - 23s 45ms/step - loss: 0.2025 - accuracy: 0.9087 - val_loss: 1.3288 - val_accuracy: 0.5885
Epoch 7/25
500/500 [==============================] - 23s 46ms/step - loss: 0.1537 - accuracy: 0.9313 - val_loss: 1.8212 - val_accuracy:

In [8]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET
max_words = 10000
word_embedding_size = 300
max_length = 15

model8 = Sequential()
model8.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512))
model8.add(Dense(1, activation='sigmoid'))

model8.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model8.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 300)           3000000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 512)           1665024   
_________________________________________________________________
lstm_5 (LSTM)                (None, 15, 512)           2099200   
_________________________________________________________________
lstm_6 (LSTM)                (None, 15, 512)           2099200   
_________________________________________________________________
lstm_7 (LSTM)                (None, 15, 512)           2099200   
_________________________________________________________________
lstm_8 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [9]:
model8.fit(x=train_padded_es_indv,
          y=y_train_es_indv,
          validation_data=(valid_padded_es_indv, y_valid_es_indv),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model8.predict(valid_padded_es_indv)
author_profiling_report(predictions)

Epoch 1/25
500/500 [==============================] - 92s 183ms/step - loss: 0.6270 - accuracy: 0.6522 - val_loss: 0.6574 - val_accuracy: 0.6274
Epoch 2/25
500/500 [==============================] - 86s 173ms/step - loss: 0.5165 - accuracy: 0.7510 - val_loss: 0.6930 - val_accuracy: 0.6159
Epoch 3/25
500/500 [==============================] - 86s 173ms/step - loss: 0.4273 - accuracy: 0.8031 - val_loss: 0.7910 - val_accuracy: 0.6131
Epoch 4/25
500/500 [==============================] - 86s 173ms/step - loss: 0.3407 - accuracy: 0.8481 - val_loss: 0.9331 - val_accuracy: 0.6043
Epoch 5/25
500/500 [==============================] - 86s 172ms/step - loss: 0.2695 - accuracy: 0.8816 - val_loss: 0.9980 - val_accuracy: 0.5949
Epoch 6/25
500/500 [==============================] - 86s 172ms/step - loss: 0.2109 - accuracy: 0.9087 - val_loss: 1.2438 - val_accuracy: 0.6014
Epoch 7/25
500/500 [==============================] - 86s 172ms/step - loss: 0.1625 - accuracy: 0.9307 - val_loss: 1.7539 - val_ac

#### 20 joined tweets

In [15]:
# ES 20 JOINED TWEETS - TRAINABLE EMB
max_words = 10000
word_embedding_size = 300
max_length = 250

model5 = Sequential()
model5.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_es_indv], # embeddings
                    input_length=max_length,
                    trainable=False,
                    mask_zero=True))
model5.add(LSTM(units=128))
model5.add(Dense(1, activation='sigmoid'))

model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 300)          3000000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 3,219,777
Trainable params: 219,777
Non-trainable params: 3,000,000
_________________________________________________________________


In [16]:
model5.fit(x=train_padded_es_20,
          y=y_train_es_20,
          validation_data=(valid_padded_es_20, y_valid_es_20),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model5.predict(valid_padded_es_20)
author_profiling_report(predictions)

Epoch 1/25
619/619 [==============================] - 20s 32ms/step - loss: 0.6615 - accuracy: 0.6042 - val_loss: 0.5851 - val_accuracy: 0.6950
Epoch 2/25
619/619 [==============================] - 19s 30ms/step - loss: 0.6424 - accuracy: 0.6290 - val_loss: 0.6069 - val_accuracy: 0.6425
Epoch 3/25
619/619 [==============================] - 18s 30ms/step - loss: 0.6345 - accuracy: 0.6349 - val_loss: 0.5698 - val_accuracy: 0.7125
Epoch 4/25
619/619 [==============================] - 19s 30ms/step - loss: 0.6293 - accuracy: 0.6421 - val_loss: 0.5821 - val_accuracy: 0.7175
Epoch 5/25
619/619 [==============================] - 18s 30ms/step - loss: 0.6235 - accuracy: 0.6478 - val_loss: 0.5911 - val_accuracy: 0.6775
Epoch 6/25
619/619 [==============================] - 18s 30ms/step - loss: 0.6182 - accuracy: 0.6535 - val_loss: 0.5549 - val_accuracy: 0.7125
Epoch 7/25
619/619 [==============================] - 18s 30ms/step - loss: 0.6116 - accuracy: 0.6577 - val_loss: 0.5936 - val_accuracy:

In [17]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB BI
max_words = 10000
word_embedding_size = 300
max_length = 250

model6 = Sequential()
model6.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model6.add(Bidirectional(LSTM(units=128, dropout=0.05, recurrent_dropout=0.2)))
model6.add(Dense(1, activation='sigmoid'))

model6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          3000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 3,439,553
Trainable params: 3,439,553
Non-trainable params: 0
_________________________________________________________________


In [18]:
model6.fit(x=train_padded_es_20,
          y=y_train_es_20,
          validation_data=(valid_padded_es_20, y_valid_es_20),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model6.predict(valid_padded_es_20)
author_profiling_report(predictions)

Epoch 1/25
619/619 [==============================] - 1095s 2s/step - loss: 0.5989 - accuracy: 0.6727 - val_loss: 0.5373 - val_accuracy: 0.7125
Epoch 2/25
619/619 [==============================] - 1095s 2s/step - loss: 0.4900 - accuracy: 0.7589 - val_loss: 0.5455 - val_accuracy: 0.7000
Epoch 3/25
619/619 [==============================] - 1093s 2s/step - loss: 0.3853 - accuracy: 0.8205 - val_loss: 0.7944 - val_accuracy: 0.6400
Epoch 4/25
619/619 [==============================] - 1093s 2s/step - loss: 0.2846 - accuracy: 0.8719 - val_loss: 0.8372 - val_accuracy: 0.6650
Epoch 5/25
619/619 [==============================] - 1091s 2s/step - loss: 0.2074 - accuracy: 0.9066 - val_loss: 1.2405 - val_accuracy: 0.6450
Epoch 6/25
619/619 [==============================] - 1095s 2s/step - loss: 0.1513 - accuracy: 0.9343 - val_loss: 1.5577 - val_accuracy: 0.6275
Epoch 7/25
619/619 [==============================] - 1121s 2s/step - loss: 0.1139 - accuracy: 0.9515 - val_loss: 1.8449 - val_accuracy:

In [10]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET
max_words = 10000
word_embedding_size = 300
max_length = 250

model9 = Sequential()
model9.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model9.add(LSTM(units=128, return_sequences=True))
model9.add(LSTM(units=128, return_sequences=True))
model9.add(LSTM(units=128, return_sequences=True))
model9.add(LSTM(units=128))
model9.add(Dense(1, activation='sigmoid'))

model9.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model9.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 300)          3000000   
_________________________________________________________________
lstm_9 (LSTM)                (None, 250, 128)          219648    
_________________________________________________________________
lstm_10 (LSTM)               (None, 250, 128)          131584    
_________________________________________________________________
lstm_11 (LSTM)               (None, 250, 128)          131584    
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,614,529
Trainable params: 3,614,529
Non-trainable params: 0
____________________________________________

In [11]:
model9.fit(x=train_padded_es_20,
          y=y_train_es_20,
          validation_data=(valid_padded_es_20, y_valid_es_20),
          shuffle=True,
          epochs=25,
          batch_size=64)

predictions = model9.predict(valid_padded_es_20)
author_profiling_report(predictions)

Epoch 1/25
619/619 [==============================] - 76s 123ms/step - loss: 0.6102 - accuracy: 0.6646 - val_loss: 0.5521 - val_accuracy: 0.7050
Epoch 2/25
619/619 [==============================] - 65s 105ms/step - loss: 0.5138 - accuracy: 0.7488 - val_loss: 0.5711 - val_accuracy: 0.7275
Epoch 3/25
619/619 [==============================] - 65s 106ms/step - loss: 0.4301 - accuracy: 0.7968 - val_loss: 0.7570 - val_accuracy: 0.6700
Epoch 4/25
619/619 [==============================] - 69s 112ms/step - loss: 0.3456 - accuracy: 0.8407 - val_loss: 0.9783 - val_accuracy: 0.6500
Epoch 5/25
619/619 [==============================] - 69s 111ms/step - loss: 0.2757 - accuracy: 0.8747 - val_loss: 1.0265 - val_accuracy: 0.6475
Epoch 6/25
619/619 [==============================] - 66s 106ms/step - loss: 0.2167 - accuracy: 0.9018 - val_loss: 1.1673 - val_accuracy: 0.6575
Epoch 7/25
619/619 [==============================] - 68s 109ms/step - loss: 0.1726 - accuracy: 0.9218 - val_loss: 1.4153 - val_ac

In [20]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET
max_words = 10000
word_embedding_size = 300
max_length = 250

model10 = Sequential()
model10.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512))
model10.add(Dense(1, activation='sigmoid'))

model10.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model10.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 300)          3000000   
_________________________________________________________________
lstm (LSTM)                  (None, 250, 512)          1665024   
_________________________________________________________________
lstm_1 (LSTM)                (None, 250, 512)          2099200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 250, 512)          2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 250, 512)          2099200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 1)                 5

In [21]:
model10.fit(x=train_padded_es_20,
          y=y_train_es_20,
          validation_data=(valid_padded_es_20, y_valid_es_20),
          shuffle=True,
          epochs=25,
          batch_size=32)

predictions = model10.predict(valid_padded_es_20)
author_profiling_report(predictions)

Epoch 1/25
1238/1238 [==============================] - 273s 221ms/step - loss: 0.6313 - accuracy: 0.6437 - val_loss: 0.6028 - val_accuracy: 0.6600
Epoch 2/25
1238/1238 [==============================] - 244s 197ms/step - loss: 0.5304 - accuracy: 0.7420 - val_loss: 0.6695 - val_accuracy: 0.6175
Epoch 3/25
1238/1238 [==============================] - 239s 193ms/step - loss: 0.4499 - accuracy: 0.7917 - val_loss: 0.6855 - val_accuracy: 0.6625
Epoch 4/25
1238/1238 [==============================] - 239s 193ms/step - loss: 0.3632 - accuracy: 0.8393 - val_loss: 0.8361 - val_accuracy: 0.6575
Epoch 5/25
1238/1238 [==============================] - 239s 193ms/step - loss: 0.2825 - accuracy: 0.8793 - val_loss: 0.9371 - val_accuracy: 0.6450
Epoch 6/25
1238/1238 [==============================] - 239s 193ms/step - loss: 0.2167 - accuracy: 0.9091 - val_loss: 1.0598 - val_accuracy: 0.6450
Epoch 7/25
1238/1238 [==============================] - 239s 193ms/step - loss: 0.1702 - accuracy: 0.9297 - val_

### Bert

In [21]:
#  ES individual tweets

bert_model1 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

bert_model1.compile(loss='binary_crossentropy',
                   optimizer=Adam(learning_rate=2e-5,epsilon=1e-08),
                   metrics=['accuracy'])
bert_model1.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bert_model1.fit(x=[train_padded_es_indv_bert, train_mask_es_20],
               y=y_train_es,
               batch_size=32,
               epochs=4,
               validation_data=([valid_padded_es_20_bert, valid_mask_es_20], y_valid_es))

predictions = bert_model1.predict([test_padded_es_20_bert, test_mask_es_20], batch_size=32)
author_profiling_report(predictions)

In [ ]:
# ES joined 20 tweets

bert_model2 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

bert_model2.compile(loss='binary_crossentropy',
                   optimizer=Adam(learning_rate=2e-5,epsilon=1e-08),
                   metrics=['accuracy'])
bert_model2.summary()

In [ ]:
bert_model2.fit(x=[train_inp, train_mask],
               y=train_label,
               batch_size=32,
               epochs=4,
               validation_data=([val_inp,val_mask],val_label))


predictions = bert_model2.predict([val_inp,val_mask],batch_size=32)
author_profiling_report(predictions)

### Classical ML

#### Train

In [22]:
# LogisticRegression

pipe = Pipeline([
        ("tfidf", TfidfVectorizer()),
        ("clf", LogisticRegression())
])
    
param_grid = {"tfidf__ngram_range" : [(1,2),(1,3),(2,3),(3,4),(3,5),(3,6),(4,5)],
              "tfidf__max_df":[0.3,0.4,0.5,0.6,0.7,0.8,0.9],
              "tfidf__min_df":[1,2,3,5], # or percentages
              "tfidf__analyzer":["char_wb"], # n-grams
              "clf__C":[1,10,100,1000,10000]
             }

clf_lr = GridSearchCV(pipe,
                      param_grid,
                      cv=5,
                      n_jobs=-1,
                      verbose=2,
                      scoring="f1_macro")

clf_lr.fit(x_train_es, y_train_es)
print(clf_lr.best_score_)
clf_lr.best_params_

Fitting 5 folds for each of 980 candidates, totalling 4900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   41.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 33.1min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 46.3min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 61.5min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 78.9min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 98.3min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 119.8min
[Parallel(n_jobs=-1)]: Done 4900 out of 4900 | elapsed: 120.3min finished


0.6550097877356043


{'clf__C': 1,
 'tfidf__analyzer': 'char_wb',
 'tfidf__max_df': 0.9,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (3, 6)}

#### Evaluation over validation

In [13]:
# Logistic Regression ES
''' BEST PARAMETERS

{'clf__C': 1,
 'tfidf__analyzer': 'char_wb',
 'tfidf__max_df': 0.9,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (3, 6)}
'''

pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(3,6),
                                 max_df=0.9,
                                 min_df=1,
                                 analyzer="char_wb")),
        ("clf", LogisticRegression(C=1))
])

pipe.fit(x_train_es, y_train_es)
predictions = pipe.predict(x_valid_es)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.76      0.80      0.78        20
        hate       0.79      0.75      0.77        20

    accuracy                           0.78        40
   macro avg       0.78      0.78      0.77        40
weighted avg       0.78      0.78      0.77        40



In [16]:
# Logistic Regression EN

pipe.fit(x_train_en, y_train_en)
predictions = pipe.predict(x_valid_en)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.76      0.80      0.78        20
        hate       0.79      0.75      0.77        20

    accuracy                           0.78        40
   macro avg       0.78      0.78      0.77        40
weighted avg       0.78      0.78      0.77        40



### Final Training

#### Auxiliar Functions

In [9]:
# Define a learning rate scheduler
def scheduler(epoch):
    if epoch < 25:
        return 0.001
    elif epoch < 50:
        return 0.0005
    elif epoch < 75:
        return 0.0001
    else:
        return 0.00005

scheduler_lr = LRS(scheduler)

In [10]:
# Optimizer
adam = Adam(learning_rate=0.001)
epochs = 100

#### Selected models

In [11]:
# ES 20 JOINED TWEETS - TRAINABLE EMB
max_words = 10000
word_embedding_size = 300
max_length = 250

model5 = Sequential()
model5.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_es_indv], # embeddings
                    input_length=max_length,
                    trainable=False,
                    mask_zero=True))
model5.add(LSTM(units=128))
model5.add(Dense(1, activation='sigmoid'))

model5.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 300)          3000000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,219,777
Trainable params: 219,777
Non-trainable params: 3,000,000
_________________________________________________________________


In [12]:
# ES INDVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET
max_words = 10000
word_embedding_size = 300
max_length = 15

model8 = Sequential()
model8.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512, return_sequences=True))
model8.add(LSTM(units=512))
model8.add(Dense(1, activation='sigmoid'))

model8.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model8.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 300)           3000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 512)           1665024   
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 512)           2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 15, 512)           2099200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 512)           2099200   
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [13]:
# ES 20 JOINED TWEETS - TRAINABLE EMB COMPLEXER NET
max_words = 10000
word_embedding_size = 300
max_length = 250

model10 = Sequential()
model10.add(Embedding(max_words,
                    output_dim=word_embedding_size,
                    #weights=[embedding_matrix_en_indv], # embeddings
                    input_length=max_length,
                    trainable=True, # Changed to True
                    mask_zero=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512, return_sequences=True))
model10.add(LSTM(units=512))
model10.add(Dense(1, activation='sigmoid'))

model10.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model10.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 300)          3000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 250, 512)          1665024   
_________________________________________________________________
lstm_7 (LSTM)                (None, 250, 512)          2099200   
_________________________________________________________________
lstm_8 (LSTM)                (None, 250, 512)          2099200   
_________________________________________________________________
lstm_9 (LSTM)                (None, 250, 512)          2099200   
_________________________________________________________________
lstm_10 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

#### Training and evaluation

##### ES 20 JOINED TWEETS - TRAINABLE EMB

In [14]:
model_5_es = model5
model_id = 'model_5_es'

model_5_es.fit(x=train_padded_es_20,
               y=y_train_es_20,
               batch_size=128,
               validation_data=(valid_padded_es_20, y_valid_es_20),
               shuffle=True,
               verbose=1,                
               epochs=epochs,
               callbacks=[configure_callbacks(model_id), scheduler_lr])

Epoch 1/100
309/310 [============================>.] - ETA: 0s - loss: 0.6624 - accuracy: 0.5996
Epoch 00001: val_accuracy improved from -inf to 0.71250, saving model to models/ap-model_5_es-0001-0.712500.h5

Epoch 00001: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 19s 61ms/step - loss: 0.6623 - accuracy: 0.5997 - val_loss: 0.6124 - val_accuracy: 0.7125
Epoch 2/100
309/310 [============================>.] - ETA: 0s - loss: 0.6444 - accuracy: 0.6243
Epoch 00002: val_accuracy improved from 0.71250 to 0.72750, saving model to models/ap-model_5_es-0002-0.727500.h5

Epoch 00002: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 18s 57ms/step - loss: 0.6443 - accuracy: 0.6243 - val_loss: 0.5708 - val_accuracy: 0.7275
Epoch 3/100
309/310 [============================>.] - ETA: 0s - loss: 0.6375 - accuracy: 0.6328
Epoch 00003: val_accuracy did not improve from 0.72750

Epoch 00003: saving model to models/ap-model_5_es.h5
3

Epoch 24/100
309/310 [============================>.] - ETA: 0s - loss: 0.4954 - accuracy: 0.7459
Epoch 00024: val_accuracy did not improve from 0.73250

Epoch 00024: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 18s 58ms/step - loss: 0.4954 - accuracy: 0.7459 - val_loss: 0.6839 - val_accuracy: 0.6850
Epoch 25/100
309/310 [============================>.] - ETA: 0s - loss: 0.4815 - accuracy: 0.7527
Epoch 00025: val_accuracy did not improve from 0.73250

Epoch 00025: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 18s 57ms/step - loss: 0.4815 - accuracy: 0.7526 - val_loss: 0.6964 - val_accuracy: 0.6950
Epoch 26/100
309/310 [============================>.] - ETA: 0s - loss: 0.4481 - accuracy: 0.7727
Epoch 00026: val_accuracy did not improve from 0.73250

Epoch 00026: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 17s 56ms/step - loss: 0.4481 - accuracy: 0.7727 - val_loss: 0.7156 - v

309/310 [============================>.] - ETA: 0s - loss: 0.1816 - accuracy: 0.9215
Epoch 00072: val_accuracy did not improve from 0.73250

Epoch 00072: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 17s 56ms/step - loss: 0.1818 - accuracy: 0.9215 - val_loss: 1.4457 - val_accuracy: 0.6675
Epoch 73/100
309/310 [============================>.] - ETA: 0s - loss: 0.1820 - accuracy: 0.9227
Epoch 00073: val_accuracy did not improve from 0.73250

Epoch 00073: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 18s 57ms/step - loss: 0.1820 - accuracy: 0.9227 - val_loss: 1.4714 - val_accuracy: 0.6650
Epoch 74/100
309/310 [============================>.] - ETA: 0s - loss: 0.1790 - accuracy: 0.9228
Epoch 00074: val_accuracy did not improve from 0.73250

Epoch 00074: saving model to models/ap-model_5_es.h5
310/310 [==============================] - 18s 58ms/step - loss: 0.1789 - accuracy: 0.9229 - val_loss: 1.4549 - val_accuracy: 

In [8]:
model_5_es = load_model('models/ap-model_5_es-0008-0.732500.h5')
predictions = model_5_es.predict(valid_padded_es_20)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.83      1.00      0.91        20
        hate       1.00      0.80      0.89        20

    accuracy                           0.90        40
   macro avg       0.92      0.90      0.90        40
weighted avg       0.92      0.90      0.90        40



##### EN 20 JOINED TWEETS - TRAINABLE EMB

In [15]:
model_5_en = model5
model_id = 'model_5_en'

model_5_en.fit(x=train_padded_en_20,
               y=y_train_es_20,
               batch_size=128,
               validation_data=(valid_padded_en_20, y_valid_en_20),
               shuffle=True,
               verbose=1,                
               epochs=epochs,
               callbacks=[configure_callbacks(model_id), scheduler_lr])

Epoch 1/100
309/310 [============================>.] - ETA: 0s - loss: 0.4000 - accuracy: 0.8213
Epoch 00001: val_accuracy improved from -inf to 0.66750, saving model to models/ap-model_5_en-0001-0.667500.h5

Epoch 00001: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 17s 56ms/step - loss: 0.3999 - accuracy: 0.8214 - val_loss: 0.9405 - val_accuracy: 0.6675
Epoch 2/100
309/310 [============================>.] - ETA: 0s - loss: 0.2911 - accuracy: 0.8713
Epoch 00002: val_accuracy improved from 0.66750 to 0.68750, saving model to models/ap-model_5_en-0002-0.687500.h5

Epoch 00002: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 18s 58ms/step - loss: 0.2912 - accuracy: 0.8713 - val_loss: 1.0512 - val_accuracy: 0.6875
Epoch 3/100
309/310 [============================>.] - ETA: 0s - loss: 0.2675 - accuracy: 0.8813
Epoch 00003: val_accuracy did not improve from 0.68750

Epoch 00003: saving model to models/ap-model_5_en.h5
3

309/310 [============================>.] - ETA: 0s - loss: 0.1348 - accuracy: 0.9470
Epoch 00024: val_accuracy did not improve from 0.68750

Epoch 00024: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 16s 52ms/step - loss: 0.1347 - accuracy: 0.9470 - val_loss: 1.6101 - val_accuracy: 0.6575
Epoch 25/100
309/310 [============================>.] - ETA: 0s - loss: 0.1241 - accuracy: 0.9510
Epoch 00025: val_accuracy did not improve from 0.68750

Epoch 00025: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 16s 52ms/step - loss: 0.1240 - accuracy: 0.9511 - val_loss: 1.7802 - val_accuracy: 0.6425
Epoch 26/100
310/310 [==============================] - ETA: 0s - loss: 0.0930 - accuracy: 0.9666
Epoch 00026: val_accuracy did not improve from 0.68750

Epoch 00026: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 16s 52ms/step - loss: 0.0930 - accuracy: 0.9666 - val_loss: 1.8407 - val_accuracy: 

310/310 [==============================] - ETA: 0s - loss: 0.0261 - accuracy: 0.9915
Epoch 00072: val_accuracy did not improve from 0.68750

Epoch 00072: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 16s 52ms/step - loss: 0.0261 - accuracy: 0.9915 - val_loss: 2.8529 - val_accuracy: 0.6275
Epoch 73/100
309/310 [============================>.] - ETA: 0s - loss: 0.0261 - accuracy: 0.9912
Epoch 00073: val_accuracy did not improve from 0.68750

Epoch 00073: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 16s 52ms/step - loss: 0.0261 - accuracy: 0.9912 - val_loss: 2.9092 - val_accuracy: 0.6150
Epoch 74/100
309/310 [============================>.] - ETA: 0s - loss: 0.0261 - accuracy: 0.9913
Epoch 00074: val_accuracy did not improve from 0.68750

Epoch 00074: saving model to models/ap-model_5_en.h5
310/310 [==============================] - 16s 52ms/step - loss: 0.0261 - accuracy: 0.9913 - val_loss: 2.8260 - val_accuracy: 

In [12]:
model_5_en = load_model('models/ap-model_5_en-0002-0.687500.h5')
predictions = model_5_en.predict(valid_padded_en_20)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.65      1.00      0.78        20
        hate       1.00      0.45      0.62        20

    accuracy                           0.73        40
   macro avg       0.82      0.72      0.70        40
weighted avg       0.82      0.72      0.70        40



##### ES INDIVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET

In [16]:
model_8_es = model8
model_id = 'model_8_es'

model_8_es.fit(x=train_padded_es_indv,
               y=y_train_es_indv,               
               batch_size=128,
               validation_data=(valid_padded_es_indv, y_valid_es_indv),
               shuffle=True,
               verbose=1,                
               epochs=epochs,
               callbacks=[configure_callbacks(model_id), scheduler_lr])

Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.6891 - accuracy: 0.5375
Epoch 00001: val_accuracy improved from -inf to 0.47462, saving model to models/ap-model_8_es-0001-0.474625.h5

Epoch 00001: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 72s 288ms/step - loss: 0.6891 - accuracy: 0.5375 - val_loss: 0.6922 - val_accuracy: 0.4746
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 0.6712 - accuracy: 0.5803
Epoch 00002: val_accuracy improved from 0.47462 to 0.53962, saving model to models/ap-model_8_es-0002-0.539625.h5

Epoch 00002: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 65s 261ms/step - loss: 0.6712 - accuracy: 0.5803 - val_loss: 0.7000 - val_accuracy: 0.5396
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.6636 - accuracy: 0.5899
Epoch 00003: val_accuracy did not improve from 0.53962

Epoch 00003: saving model to models/ap-model_8_es.h5

250/250 [==============================] - ETA: 0s - loss: 0.0304 - accuracy: 0.9882
Epoch 00047: val_accuracy did not improve from 0.61387

Epoch 00047: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 63s 254ms/step - loss: 0.0304 - accuracy: 0.9882 - val_loss: 2.8502 - val_accuracy: 0.5771
Epoch 48/100
250/250 [==============================] - ETA: 0s - loss: 0.0271 - accuracy: 0.9896
Epoch 00048: val_accuracy did not improve from 0.61387

Epoch 00048: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 65s 260ms/step - loss: 0.0271 - accuracy: 0.9896 - val_loss: 2.8672 - val_accuracy: 0.5739
Epoch 49/100
250/250 [==============================] - ETA: 0s - loss: 0.0313 - accuracy: 0.9879
Epoch 00049: val_accuracy did not improve from 0.61387

Epoch 00049: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 65s 260ms/step - loss: 0.0313 - accuracy: 0.9879 - val_loss: 2.7672 - val_accurac

250/250 [==============================] - ETA: 0s - loss: 0.0039 - accuracy: 0.9975
Epoch 00095: val_accuracy did not improve from 0.61387

Epoch 00095: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 65s 258ms/step - loss: 0.0039 - accuracy: 0.9975 - val_loss: 4.4085 - val_accuracy: 0.5734
Epoch 96/100
250/250 [==============================] - ETA: 0s - loss: 0.0039 - accuracy: 0.9974
Epoch 00096: val_accuracy did not improve from 0.61387

Epoch 00096: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 66s 265ms/step - loss: 0.0039 - accuracy: 0.9974 - val_loss: 4.4580 - val_accuracy: 0.5746
Epoch 97/100
250/250 [==============================] - ETA: 0s - loss: 0.0039 - accuracy: 0.9972
Epoch 00097: val_accuracy did not improve from 0.61387

Epoch 00097: saving model to models/ap-model_8_es.h5
250/250 [==============================] - 66s 264ms/step - loss: 0.0039 - accuracy: 0.9972 - val_loss: 4.4149 - val_accurac

In [13]:
model_8_es = load_model('models/ap-model_8_es-0010-0.613875.h5')
predictions = model_8_es.predict(valid_padded_es_indv)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.87      0.65      0.74        20
        hate       0.72      0.90      0.80        20

    accuracy                           0.78        40
   macro avg       0.79      0.78      0.77        40
weighted avg       0.79      0.78      0.77        40



##### EN INDIVIDUAL TWEETS - TRAINABLE EMB COMPLEXER NET

In [17]:
model_8_en = model8
model_id = 'model_8_en'

model_8_en.fit(x=train_padded_en_indv,
               y=y_train_en_indv,               
               batch_size=128,
               validation_data=(valid_padded_en_indv, y_valid_en_indv),
               shuffle=True,
               verbose=1,                
               epochs=epochs,
               callbacks=[configure_callbacks(model_id), scheduler_lr])

Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.2226 - accuracy: 0.9166
Epoch 00001: val_accuracy improved from -inf to 0.58025, saving model to models/ap-model_8_en-0001-0.580250.h5

Epoch 00001: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 65s 259ms/step - loss: 0.2226 - accuracy: 0.9166 - val_loss: 1.3019 - val_accuracy: 0.5803
Epoch 2/100
250/250 [==============================] - ETA: 0s - loss: 0.1390 - accuracy: 0.9500
Epoch 00002: val_accuracy did not improve from 0.58025

Epoch 00002: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 65s 259ms/step - loss: 0.1390 - accuracy: 0.9500 - val_loss: 1.6027 - val_accuracy: 0.5742
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.0960 - accuracy: 0.9654
Epoch 00003: val_accuracy improved from 0.58025 to 0.58300, saving model to models/ap-model_8_en-0003-0.583000.h5

Epoch 00003: saving model to models/ap-model_8_en.h5

Epoch 24/100
250/250 [==============================] - ETA: 0s - loss: 0.0186 - accuracy: 0.9933
Epoch 00024: val_accuracy did not improve from 0.58300

Epoch 00024: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 65s 258ms/step - loss: 0.0186 - accuracy: 0.9933 - val_loss: 2.9177 - val_accuracy: 0.5792
Epoch 25/100
250/250 [==============================] - ETA: 0s - loss: 0.0196 - accuracy: 0.9931
Epoch 00025: val_accuracy did not improve from 0.58300

Epoch 00025: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 66s 265ms/step - loss: 0.0196 - accuracy: 0.9931 - val_loss: 2.6301 - val_accuracy: 0.5798
Epoch 26/100
250/250 [==============================] - ETA: 0s - loss: 0.0100 - accuracy: 0.9957
Epoch 00026: val_accuracy did not improve from 0.58300

Epoch 00026: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 66s 264ms/step - loss: 0.0100 - accuracy: 0.9957 - val_loss: 3.0834 

250/250 [==============================] - ETA: 0s - loss: 0.0052 - accuracy: 0.9969
Epoch 00048: val_accuracy did not improve from 0.58300

Epoch 00048: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 64s 258ms/step - loss: 0.0052 - accuracy: 0.9969 - val_loss: 4.0123 - val_accuracy: 0.5790
Epoch 49/100
250/250 [==============================] - ETA: 0s - loss: 0.0114 - accuracy: 0.9950
Epoch 00049: val_accuracy did not improve from 0.58300

Epoch 00049: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 66s 264ms/step - loss: 0.0114 - accuracy: 0.9950 - val_loss: 3.4569 - val_accuracy: 0.5778
Epoch 50/100
250/250 [==============================] - ETA: 0s - loss: 0.0101 - accuracy: 0.9958
Epoch 00050: val_accuracy did not improve from 0.58300

Epoch 00050: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 66s 263ms/step - loss: 0.0101 - accuracy: 0.9958 - val_loss: 3.5321 - val_accurac

250/250 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 0.9977
Epoch 00096: val_accuracy did not improve from 0.58300

Epoch 00096: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 65s 260ms/step - loss: 0.0031 - accuracy: 0.9977 - val_loss: 4.4024 - val_accuracy: 0.5815
Epoch 97/100
250/250 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 0.9974
Epoch 00097: val_accuracy did not improve from 0.58300

Epoch 00097: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 66s 266ms/step - loss: 0.0031 - accuracy: 0.9974 - val_loss: 4.4117 - val_accuracy: 0.5803
Epoch 98/100
250/250 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 0.9977
Epoch 00098: val_accuracy did not improve from 0.58300

Epoch 00098: saving model to models/ap-model_8_en.h5
250/250 [==============================] - 65s 260ms/step - loss: 0.0031 - accuracy: 0.9977 - val_loss: 4.4111 - val_accurac

In [14]:
model_8_en = load_model('models/ap-model_8_en-0003-0.583000.h5')
predictions = model_8_en.predict(valid_padded_en_indv)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.83      0.75      0.79        20
        hate       0.77      0.85      0.81        20

    accuracy                           0.80        40
   macro avg       0.80      0.80      0.80        40
weighted avg       0.80      0.80      0.80        40



##### ES 20 JOINED TWEETS - TRAINABLE EMB COMPLEXER NET

In [ ]:
model_10_es = model10
model_id = 'model_10_es'

model_10_es.fit(x=train_padded_es_20,
                y=y_train_es_20,
                batch_size=128,
                validation_data=(valid_padded_es_20, y_valid_es_20),
                shuffle=True,
                verbose=1,                
                epochs=epochs,
                callbacks=[configure_callbacks(model_id), scheduler_lr])

In [15]:
model_10_es = load_model('models/ap-model_10_es-0002-0.727500.h5')
predictions = model_10_es.predict(valid_padded_es_20)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.67      1.00      0.80        20
        hate       1.00      0.50      0.67        20

    accuracy                           0.75        40
   macro avg       0.83      0.75      0.73        40
weighted avg       0.83      0.75      0.73        40



##### EN 20 JOINED TWEETS - TRAINABLE EMB COMPLEXER NET

In [ ]:
model_10_en = model10
model_id = 'model_10_en'

model_10_en.fit(x=train_padded_en_20,
                y=y_train_en_20,
                batch_size=128
                validation_data=(valid_padded_en_20, y_valid_en_20),
                shuffle=True,
                verbose=1,                
                epochs=epochs,
                callbacks=[configure_callbacks(model_id), scheduler])

In [16]:
model_10_en = load_model('models/ap-model_10_en-0078-0.660000.h5')
predictions = model_10_en.predict(valid_padded_en_20)
author_profiling_report(predictions)

              precision    recall  f1-score   support

    not hate       0.65      1.00      0.78        20
        hate       1.00      0.45      0.62        20

    accuracy                           0.73        40
   macro avg       0.82      0.72      0.70        40
weighted avg       0.82      0.72      0.70        40



### Test predictions

In [11]:
# Logistic Regression

pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(3,6),
                                 max_df=0.9,
                                 min_df=1,
                                 analyzer="char_wb")),
        ("clf", LogisticRegression(C=1))
])

pipe.fit(x_train_es, y_train_es)
predictions = pipe.predict(x_test_es)
author_profiling_report(predictions, number_authors=100)

pipe.fit(x_train_en, y_train_en)
predictions = pipe.predict(x_test_en)
author_profiling_report(predictions, number_authors=100)

              precision    recall  f1-score   support

    not hate       0.43      0.38      0.40        50
        hate       0.45      0.50      0.47        50

    accuracy                           0.44       100
   macro avg       0.44      0.44      0.44       100
weighted avg       0.44      0.44      0.44       100

              precision    recall  f1-score   support

    not hate       0.43      0.38      0.40        50
        hate       0.45      0.50      0.47        50

    accuracy                           0.44       100
   macro avg       0.44      0.44      0.44       100
weighted avg       0.44      0.44      0.44       100



array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0])

In [12]:
# Model 5 indv
model_5_es = load_model('models/ap-model_5_es-0008-0.732500.h5')
predictions = model_5_es.predict(test_padded_es_20)
author_profiling_report(predictions, number_authors=100)

model_5_en = load_model('models/ap-model_5_en-0002-0.687500.h5')
predictions = model_5_en.predict(test_padded_en_20)
author_profiling_report(predictions, number_authors=100)

              precision    recall  f1-score   support

    not hate       0.50      0.16      0.24        50
        hate       0.50      0.84      0.63        50

    accuracy                           0.50       100
   macro avg       0.50      0.50      0.43       100
weighted avg       0.50      0.50      0.43       100

              precision    recall  f1-score   support

    not hate       0.55      0.48      0.51        50
        hate       0.54      0.60      0.57        50

    accuracy                           0.54       100
   macro avg       0.54      0.54      0.54       100
weighted avg       0.54      0.54      0.54       100



array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1])

In [13]:
# Model 8 20 joined
model_8_es = load_model('models/ap-model_8_es-0010-0.613875.h5')
predictions = model_8_es.predict(test_padded_en_indv)
author_profiling_report(predictions, number_authors=100)

model_8_en = load_model('models/ap-model_8_en-0003-0.583000.h5')
predictions = model_8_en.predict(test_padded_en_indv)
author_profiling_report(predictions, number_authors=100)

              precision    recall  f1-score   support

    not hate       0.51      0.86      0.64        50
        hate       0.56      0.18      0.27        50

    accuracy                           0.52       100
   macro avg       0.54      0.52      0.46       100
weighted avg       0.54      0.52      0.46       100

              precision    recall  f1-score   support

    not hate       0.48      0.66      0.55        50
        hate       0.45      0.28      0.35        50

    accuracy                           0.47       100
   macro avg       0.46      0.47      0.45       100
weighted avg       0.46      0.47      0.45       100



array([1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [14]:
pickle_file = open('test_es_20.pickle', 'rb')
test_es_20 = pickle.load(pickle_file)

test_id_es, x_test_es = test_es_20[0], test_es_20[1]

pickle_file = open('test_en_20.pickle', 'rb')
test_en_20 = pickle.load(pickle_file)

test_id_en, x_test_en = test_en_20[0], test_en_20[1]

In [15]:
# Model 5 indv
model_5_es = load_model('models/ap-model_5_es-0008-0.732500.h5')
predictions_es = model_5_es.predict(x_test_es)

model_5_en = load_model('models/ap-model_5_en-0002-0.687500.h5')
predictions_en = model_5_en.predict(x_test_en)

In [16]:
len(x_test_en), len(predictions_es)

(1000, 1000)

In [17]:
predictions_es = author_profiling_report(predictions_es, number_authors=100)
predictions_en = author_profiling_report(predictions_en, number_authors=100)

              precision    recall  f1-score   support

    not hate       0.50      0.16      0.24        50
        hate       0.50      0.84      0.63        50

    accuracy                           0.50       100
   macro avg       0.50      0.50      0.43       100
weighted avg       0.50      0.50      0.43       100

              precision    recall  f1-score   support

    not hate       0.55      0.48      0.51        50
        hate       0.54      0.60      0.57        50

    accuracy                           0.54       100
   macro avg       0.54      0.54      0.54       100
weighted avg       0.54      0.54      0.54       100



In [21]:
aux = []
for i in range(0, len(test_id_es), 10):
    aux.append(test_id_es[i])
test_id_es = aux
len(test_id_es), len(predictions_es)

(100, 100)

In [23]:
aux = []
for i in range(0, len(test_id_en), 10):
    aux.append(test_id_en[i])
test_id_en = aux
len(test_id_en), len(predictions_en)

(100, 100)

In [18]:
# 100 authors 100 tweets in test
# txt output
# <author id="author-id" lang="en|es" type="0|1"/>
def format_output(author_predictions, authors, lang):
    for i in range(len(author_predictions)):
        root = ET.Element("author")
        root.set("id", str(authors[i]))
        root.set("lang", lang)
        root.set("type", str(author_predictions[i]))

        tree = ET.ElementTree(root)
        xml_str = ET.tostring(root).decode('utf8')

        save_path_file = f'results/{lang}/{authors[i]}.xml'

        with open(save_path_file, "w") as f:
            f.write(xml_str)

In [24]:
format_output(predictions_es, test_id_es, "es")
format_output(predictions_en, test_id_en, "en")